## CAKE experiment on HB 

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, average_precision_score
from dataset import Dataset
from myModel import MyModel, MyDataset
from myExplainers import MyExplainer
from myEvaluation import MyEvaluation
import pickle
from tqdm import tqdm
import datetime
import csv
import warnings
import torch
import tensorflow as tf
from scipy.special import softmax
from helper import print_results
from cake import CAKE

In [2]:
torch.cuda.empty_cache() 

Load model, data and task

In [4]:
data_path = ''
model_path = 'Trained Models/'
save_path = '/home/myloniko/ethos/Results/HB/'

In [5]:
model_name = 'bert'
existing_rationales = True

In [6]:
task = 'multi_label'
labels = 6
model = MyModel(model_path, 'bert_hummingbird', model_name, task, labels, False)
max_sequence_len = model.tokenizer.max_len_single_sentence
tokenizer = model.tokenizer
import torch
torch.cuda.is_available()
model.trainer.model.to('cuda')

BertForMultilabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [7]:
hb = Dataset(path = data_path)
x, y, label_names, rationales = hb.load_hummingbird()

/home/myloniko/ethos/dataset.py:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  z = np.array(z)


Split data

In [8]:
indices = np.arange(len(y))
train_texts, test_texts, train_labels, test_labels, _, test_indexes = train_test_split(x, y,  indices, test_size=.2, random_state=42)
if existing_rationales:
    test_rationales = [rationales[x] for x in test_indexes]

size = (0.1 * len(y)) / len(train_labels)
train_texts, validation_texts, train_labels, validation_labels = train_test_split(list(train_texts), train_labels, test_size=size, random_state=42)

In [9]:
new_rationale = []
for i in range(100):
    rationale = []
    test_t = test_texts[i].split(' ')
    for j in range(6):
        label_rational = []
        for k in range(len(test_t)):
            for r in tokenizer.tokenize(test_t[k]):
                rationall = 1 if test_rationales[i][j][k] > 0 else 0
                label_rational.append(rationall)
        rationale.append(label_rational)
    new_rationale.append(rationale)

In [ ]:
predictions = []
for test_text in test_texts:
    outputs = model.my_predict(test_text)
    predictions.append(outputs[0])

a = tf.constant(predictions, dtype = tf.float32)
b = tf.keras.activations.sigmoid(a)
predictions = b.numpy()

In [11]:
pred_labels = []
for prediction in predictions:
    pred_labels.append([1 if i >= 0.5 else 0 for i in prediction])

def average_precision_wrapper(y, y_pred, view):
    return average_precision_score(y, y_pred.toarray(), average=view)

print(average_precision_score(test_labels, pred_labels, average='macro'), f1_score(test_labels, pred_labels, average='macro'))

0.4871469706211206 0.6110325760338523


In [12]:
label_names

['anger', 'disgust', 'fear', 'joy', 'offensive', 'sadness']

Define the label descriptions

In [13]:
description = ["Anger label: is a complex emotion that can manifest in a range of ways in text, such as through the use of strong, forceful language or aggressive and confrontational statements.",
               "Disgust label: is typically associated with aversion or revulsion towards something that is perceived as unpleasant or offensive, such as offensive language or imagery. In text, disgust can be conveyed through a variety of linguistic cues, such as the use of negative adjectives, references to bodily fluids or waste, or explicit expressions of revulsion.",
               "Fear label: is a complex emotion that is typically associated with a perceived threat or danger. In text, fear can be conveyed through a variety of linguistic cues, such as the use of negative adjectives, references to danger or risk, or explicit expressions of fear or anxiety. ",
               "Joy label: is typically associated with positive experiences, such as happiness, amusement, or pleasure. In text, joy can be conveyed through a variety of linguistic cues, such as the use of positive adjectives, references to pleasant experiences, or explicit expressions of happiness or enjoyment. ",
               "Offensive label: concerns language that can be conveyed through a variety of linguistic cues, such as the use of derogatory terms, references to marginalized groups, or explicit expressions of hate or animosity. ",
               "Sadness label: is typically associated with negative experiences, such as loss, grief, or disappointment. In text, sadness can be conveyed through a variety of linguistic cues, such as the use of negative adjectives, references to loss or grief, or explicit expressions of sadness or despair."
]
len(description)

6

Create a small cake (CAKE's instance)

In [14]:
cake = CAKE(model_path = 'Trained Models/bert_hummingbird', tokenizer = tokenizer, label_names = label_names, 
            label_descriptions = description, input_docs = train_texts, input_labels = list(train_labels), 
            input_docs_test = list(test_texts))

In [17]:
my_explainers = MyExplainer(label_names, model, cake=cake)

my_evaluators = MyEvaluation(label_names, model.my_predict, False, True)
my_evaluatorsP = MyEvaluation(label_names, model.my_predict, False, False)
evaluation =  {'F':my_evaluators.faithfulness, 'FTP': my_evaluators.faithful_truthfulness_penalty, 
          'NZW': my_evaluators.nzw, 'AUPRC': my_evaluators.auprc}
evaluationP = {'F':my_evaluatorsP.faithfulness, 'FTP': my_evaluatorsP.faithful_truthfulness_penalty, 
          'NZW': my_evaluatorsP.nzw, 'AUPRC': my_evaluators.auprc}

In [27]:
confs = []
for key_emb in [1, 2, 3]:
    for label_emb in [1, 2, "2_doc", 3]:
        for keyphrases in [5, 10, 15, 20]:
            for width in [0, 1, 2, 3, 5]:
                for negatives in [False]:
                    confs.append([key_emb, label_emb, keyphrases, width, negatives])
len(confs)

180

In [ ]:
import time
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    file_name = save_path + 'HB_BERT_CAKEZ_NEW_20'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)
    metrics = {'F':[], 'FTP':[], 'AUPRC': [], 'NZW':[]}
    metricsP = {'F':[], 'FTP':[], 'AUPRC': [], 'NZW':[]}
    time_r = []
    for conf in confs:
        time_r.append([])
    techniques = [my_explainers.cake_explain] 
    for ind in tqdm(range(0,len(test_texts))):
        torch.cuda.empty_cache() 
        test_rational = new_rationale[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        prediction, _, _ = model.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
    
        interpretations = []
        kk = 0
        for conf in confs:
            ts = time.time()
            if conf[1] == 3:
                my_explainers.cake_conf = [conf[0], conf[1], ind, conf[2], conf[3], conf[4]]
            else:
                my_explainers.cake_conf = [conf[0], conf[1], None, conf[2], conf[3], conf[4]]
            temp = techniques[0](instance, prediction, tokens, mask, _, _)
            interpretations.append([np.array(i)/np.max(np.abs(i)) if np.max(np.abs(i))!=0 else np.zeros(len(i)) for i in temp])
            time_r[kk].append(time.time()-ts)
            kk = kk + 1
        for metric in metrics.keys():
            evaluated = []
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
            metrics[metric].append(evaluated)
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()
        my_evaluators.clear_states()
        for metric in metrics.keys():
            evaluatedP = []
            for interpretation in interpretations:
                evaluatedP.append(evaluationP[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
            metricsP[metric].append(evaluatedP)
        with open(file_name+'(A).pickle', 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'(P).pickle', 'wb') as handle:
            pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'_TIME.pickle', 'wb') as handle:
            pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
time_r = np.array(time_r)
time_r.mean(axis=1)

In [ ]:
print_results(file_name+'(P)', confs, metricsP, label_names)

# Time analysis

In [22]:
confs = []
for key_emb in [1, 2, 3]:
    for label_emb in [1, 2, 3]:
        for keyphrases in [5, 10, 15, 20]:
            for width in [0, 1, 2, 3]:
                for negatives in [False]:
                    confs.append([key_emb, label_emb, keyphrases, width, negatives])
len(confs)

144

In [ ]:
import time
from tqdm.notebook import tqdm

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    time_r = []
    for conf in confs:
        time_r.append([])
    techniques = [my_explainers.cake_explain] 
    for ind in tqdm(range(0,10)):
        torch.cuda.empty_cache() 
        test_rational = new_rationale[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        prediction, _, _ = model.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
    
        interpretations = []
        kk = 0
        for conf in tqdm(confs):
            ts = time.time()
            if conf[1] == 3:
                my_explainers.cake_conf = [conf[0], conf[1], ind, conf[2], conf[3], conf[4]]
            else:
                my_explainers.cake_conf = [conf[0], conf[1], None, conf[2], conf[3], conf[4]]
            temp = techniques[0](instance, prediction, tokens, mask, _, _)
            aa = [np.array(i)/np.max(np.abs(i)) if np.max(np.abs(i))!=0 else np.zeros(len(i)) for i in temp]
            time_r[kk].append(time.time()-ts)
            kk = kk + 1
time_r = np.array(time_r)
time_r.mean(axis=1)

In [24]:
list(zip(confs,list(time_r.mean(axis=1))))

[([1, 1, 5, 0, False], 0.13995487689971925),
 ([1, 1, 5, 1, False], 0.12349448204040528),
 ([1, 1, 5, 2, False], 0.12362327575683593),
 ([1, 1, 5, 3, False], 0.12502355575561525),
 ([1, 1, 10, 0, False], 0.12095119953155517),
 ([1, 1, 10, 1, False], 0.12590150833129882),
 ([1, 1, 10, 2, False], 0.12279009819030762),
 ([1, 1, 10, 3, False], 0.11835381984710694),
 ([1, 1, 15, 0, False], 0.12001774311065674),
 ([1, 1, 15, 1, False], 0.1228823184967041),
 ([1, 1, 15, 2, False], 0.11905977725982667),
 ([1, 1, 15, 3, False], 0.11908559799194336),
 ([1, 1, 20, 0, False], 0.11889691352844238),
 ([1, 1, 20, 1, False], 0.1221503734588623),
 ([1, 1, 20, 2, False], 0.11828882694244384),
 ([1, 1, 20, 3, False], 0.1198040246963501),
 ([1, 2, 5, 0, False], 0.11802587509155274),
 ([1, 2, 5, 1, False], 0.11724326610565186),
 ([1, 2, 5, 2, False], 0.11751015186309814),
 ([1, 2, 5, 3, False], 0.12171969413757325),
 ([1, 2, 10, 0, False], 0.1192744493484497),
 ([1, 2, 10, 1, False], 0.11874027252197265),
